In [65]:
pip install pygame

Python(3723) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [66]:
import pandas as pd
import numpy as np
import pygame
import math
import random
from IPython.display import display, HTML

<h2> Parameters </h2>

In [67]:
SIDE_LEN = 99
START_SQAURE = [49, 49]
GOAL_SQAURE = [10, 90]
REWARD_INDEX = 1000
LEARNING_RATE = .9
DISCOUNT_FACTOR = .75
MAX_RUNTIME = SIDE_LEN**2


<H1> Initialize Environment </h1>
Set up the board the agent will work with, plus the randomized start/finish blocks

In [68]:
gameBoard = [[0 for i in range(SIDE_LEN)] for j in range(SIDE_LEN)]
rewardBoard = [[0 for i in range(SIDE_LEN)] for j in range(SIDE_LEN)]



CurrSquare = START_SQAURE.copy()

def upMove(curr):
    # gameBoard[curr[0]][curr[1]] = 0
    if curr[0] == 0:
        return
    curr[0] = curr[0]-1
    # gameBoard[curr[0]][curr[1]] = 1
    reward = calcReward(curr)
    rewardBoard[curr[0]][curr[1]] = reward
    
def downMove(curr):
    # gameBoard[curr[0]][curr[1]] = 0
    if curr[0] == SIDE_LEN-1:
        return
    curr[0] = curr[0]+1
    # gameBoard[curr[0]][curr[1]] = 1
    reward = calcReward(curr)
    rewardBoard[curr[0]][curr[1]] = reward
    
def leftMove(curr):
    # gameBoard[curr[0]][curr[1]] = 0
    if curr[1] == 0:
        return
    curr[1] = curr[1]-1
    # gameBoard[curr[0]][curr[1]] = 1
    reward = calcReward(curr)
    rewardBoard[curr[0]][curr[1]] = reward

def rightMove(curr):
    # gameBoard[curr[0]][curr[1]] = 0
    if curr[1] == SIDE_LEN-1:
        return
    curr[1] = curr[1]+1   
    # gameBoard[curr[0]][curr[1]] = 1
    reward = calcReward(curr)
    rewardBoard[curr[0]][curr[1]] = reward

def calcDist(curr):
    one = curr[0] - GOAL_SQAURE[0]
    two = curr[1] - GOAL_SQAURE[1]
    return math.sqrt(pow(one, 2) + pow(two, 2))


START_BASE_REWARD = round(REWARD_INDEX/calcDist(START_SQAURE), 2)

def calcReward(curr):
    if curr == GOAL_SQAURE:
        return REWARD_INDEX**2
    else:
        distance = calcDist(curr)
        reward = round(REWARD_INDEX/distance, 2) - START_BASE_REWARD
        reward = reward*REWARD_INDEX
        return round(reward, 2)



In [69]:


# upMove(CurrSquare)

# for row in gameBoard:
#     print(row)
    
# print("")
# print(" - - - - - - - - - - - ")
# print("")

# for row in rewardBoard:
#     print(row)
    
# print("")
# print(" - - - - - - - - - - - ")
# print("")

# print(CurrSquare)

<h1> Initialize SAV Table and Q-Table</h1>

State-Action-Value table: <br>
    Each Row is a state<br>
    Each Column is a different action<br>
    Each Cell is the reard for taking that columns action after starting in the row's state<br>
    Final Column is Best Action<br>
    

In [70]:
SAV = pd.DataFrame(-1.0, index = np.arange(SIDE_LEN**2), columns=["UP", "DOWN", "LEFT", "RIGHT"])
SAV['MAX'] = 0.0
TOP_SAV = pd.DataFrame('na', index = np.arange(SIDE_LEN**2), columns = ['BEST'])
TOP_SAV['BIDX'] = -1

def updateSAV(prev, move, reward, curr):
    idx = prev[0]*11 + prev[1]
    curridx = curr[0]*11 + curr[1]
    SAV.at[idx, move] = reward
    SAV['MAX'] = SAV.max(axis=1)
    max_action = SAV.loc[idx, ['UP', 'DOWN', 'LEFT', 'RIGHT']].idxmax()
    TOP_SAV.at[idx, 'BEST'] = max_action
    TOP_SAV.at[idx, 'BIDX'] = calc_Max_Idx(idx, max_action)
    
def calc_Max_Idx(currIdx, action):
    if action == 'UP':
        maxidx = currIdx - SIDE_LEN
        if maxidx < 0:
            maxidx = currIdx + SIDE_LEN
        if maxidx >= SIDE_LEN**2:
            maxidx = currIdx - SIDE_LEN
        return maxidx
    
    if action == 'DOWN':
        maxidx = currIdx + SIDE_LEN
        if maxidx < 0:
            maxidx = currIdx + SIDE_LEN
        if maxidx >= SIDE_LEN**2:
            maxidx = currIdx - SIDE_LEN
        return maxidx
    
    if action == 'LEFT':
        maxidx = currIdx - 1
        if maxidx < 0:
            maxidx = currIdx + SIDE_LEN
        if maxidx >= SIDE_LEN**2:
            maxidx = currIdx - SIDE_LEN
        return maxidx
    
    if action == 'RIGHT':
        maxidx = currIdx + 1
        if maxidx < 0:
            maxidx = currIdx + SIDE_LEN
        if maxidx >= SIDE_LEN**2:
            maxidx = currIdx - SIDE_LEN
        return maxidx

        
Q_TABLE = pd.DataFrame(0.0, index = np.arange(SIDE_LEN**2), columns=["UP", "DOWN", "LEFT", "RIGHT", "MAX"])
TOP_Q = pd.DataFrame('na', index = np.arange(SIDE_LEN**2), columns = ['BEST'])
TOP_Q['BIDX'] = -1

def bellmanEquation(currQ, reward, stateIdx, df, recurse):
    if df.at[stateIdx, 'BEST'] == 'na':
        return currQ + (LEARNING_RATE * (reward + DISCOUNT_FACTOR * (0))) - currQ
    else:
        if recurse == 20:
            return currQ + (LEARNING_RATE * (reward + DISCOUNT_FACTOR * (0))) - currQ
        recurse += 1
        nextIdx = df.at[stateIdx, 'BIDX']
        nextQ = Q_TABLE.at[nextIdx, 'MAX']
        nextReward = SAV.at[nextIdx, 'MAX']

        return currQ +(LEARNING_RATE * (reward + DISCOUNT_FACTOR * (bellmanEquation(nextQ, nextReward, nextIdx, df, recurse)))) - currQ
    
def updateQTable(prev, move, reward, curr):
    idx = prev[0]*11 + prev[1]
    curridx = curr[0]*11 + curr[1]
    if idx != TOP_Q.at[curridx, 'BIDX']:
        Q_TABLE.at[idx, move] = bellmanEquation(Q_TABLE.at[idx, move], reward, idx, TOP_Q, 0)
    Q_TABLE['MAX'] = Q_TABLE.max(axis=1)
    max_action = Q_TABLE.loc[idx, ['UP', 'DOWN', 'LEFT', 'RIGHT']].idxmax()
    TOP_Q.at[idx, 'BEST'] = max_action
    TOP_Q.at[idx, 'BIDX'] = calc_Max_Idx(idx, max_action)

    

In [71]:
def move(direction, curr):
    prev = curr.copy()
    match direction:
        case "UP":
            upMove(curr)
        case "DOWN":
            downMove(curr)
        case "LEFT":
            leftMove(curr)
        case "RIGHT":
            rightMove(curr)
        case _:
            print("INVALID MOVE ERROR")
            exit(99)
    if curr == prev:
        reward = -1 * (REWARD_INDEX**2)
    else:
        reward = rewardBoard[curr[0]][curr[1]]
    # print(prev)
    # print(curr)
    updateSAV(prev, direction, reward, curr)
    updateQTable(prev, direction, reward, curr)
    

# move("UP", CurrSquare)
# print(SAV.loc[60,:])
# print(TOP_SAV.loc[60,:])
# print(Q_TABLE.loc[60,:])
# print(TOP_Q.loc[60,:])
    

<h1> Agent Time </h1>

In [72]:
def runAgent(curr, iter, randChance):
    moves = 0
    while moves < MAX_RUNTIME:
        rand = random.randint(1,100)
        currIdx = curr[0]*11 + curr[1]
        if rand <= randChance:
            direct = SAV.loc[currIdx][SAV.loc[currIdx] == -1.0].index.to_list()
            # direct = ["UP", "DOWN", "LEFT", "RIGHT"]
            if len(direct) == 0:
                direction = TOP_Q.at[currIdx, 'BEST']
                if direction == 'na':
                    direction = random.choice(["UP", "DOWN", "LEFT", "RIGHT"])
            else:
                direction = random.choice(direct)
        else:
            
            direction = TOP_Q.at[currIdx, 'BEST']
            if direction == 'na':
                direct = SAV.loc[currIdx][SAV.loc[currIdx] == -1.0].index.to_list()
                if len(direct) == 0:
                    direction = random.choice(["UP", "DOWN", "LEFT", "RIGHT"])
                else:
                    direction = random.choice(direct)
        prev = curr.copy()
        move(direction, curr)
        if curr == prev:
            print("Hit an edge")
            break
        if curr == GOAL_SQAURE:
            print("REACHED GOAL!")
            break
        moves += 1
        
    print(f"Iter: {iter} - Stopped after {moves} moves.")
    print("") 

In [73]:
MAX_ITER = 1000
CurrSqaure = START_SQAURE

for i in range(MAX_ITER):
    runAgent(CurrSquare, i, 100-(i/10))
    gameBoard = [[0 for i in range(SIDE_LEN)] for j in range(SIDE_LEN)]
    CurrSquare = START_SQAURE.copy()
    


Iter: 0 - Stopped after 9801 moves.

Iter: 1 - Stopped after 9801 moves.

Iter: 2 - Stopped after 9801 moves.

Iter: 3 - Stopped after 9801 moves.

Iter: 4 - Stopped after 9801 moves.

Iter: 5 - Stopped after 9801 moves.

Iter: 6 - Stopped after 9801 moves.

Iter: 7 - Stopped after 9801 moves.

Iter: 8 - Stopped after 9801 moves.

Iter: 9 - Stopped after 9801 moves.

Iter: 10 - Stopped after 9801 moves.

Iter: 11 - Stopped after 9801 moves.

Iter: 12 - Stopped after 9801 moves.

Iter: 13 - Stopped after 9801 moves.

Iter: 14 - Stopped after 9801 moves.

Iter: 15 - Stopped after 9801 moves.

Iter: 16 - Stopped after 9801 moves.

Iter: 17 - Stopped after 9801 moves.

Iter: 18 - Stopped after 9801 moves.

Iter: 19 - Stopped after 9801 moves.

Iter: 20 - Stopped after 9801 moves.

Iter: 21 - Stopped after 9801 moves.

Iter: 22 - Stopped after 9801 moves.

Iter: 23 - Stopped after 9801 moves.

Iter: 24 - Stopped after 9801 moves.

Iter: 25 - Stopped after 9801 moves.

Iter: 26 - Stopped aft

KeyboardInterrupt: 

In [ ]:
# display(HTML(TOP_Q.to_html()))
# for row in rewardBoard:
#     print(row)


,BEST,BIDX
0,na,-1
1,na,-1
2,na,-1
3,na,-1
4,DOWN,15
5,UP,16
6,RIGHT,7
7,UP,18
8,DOWN,19
9,na,-1


[0, 0, 0, 0, 193.0, 657.0, 1394.0, 2704.0, 5303.0, 0, 0]
[0, 0, 0, 0, 232.0, 732.0, 1565.0, 3232.0, 8232.0, 10000, 8232.0]
[0, 0, 0, 0, 0, 657.0, 1394.0, 2704.0, 5303.0, 8232.0, 5303.0]
[0, 0, 0, 0, 89.0, 468.0, 1006.0, 1768.0, 2704.0, 3232.0, 0]
[0, 0, 0, 0, -53.0, 232.0, 589.0, 1006.0, 1394.0, 1565.0, 1394.0]
[0, 0, 0, -381.0, -206.0, 0.0, 232.0, 468.0, 657.0, 732.0, 0]
[0, 0, 0, -488.0, -354.0, -206.0, -53.0, 89.0, 193.0, 232.0, 0]
[0, 0, -683.0, -589.0, -488.0, -381.0, 0, 0, -124.0, -101.0, -124.0]
[0, 0, 0, -683.0, -606.0, 0, -455.0, -394.0, -354.0, 0, 0]
[0, 0, 0, -768.0, -708.0, -650.0, -598.0, -555.0, -528.0, 0, 0]
[-982.0, -938.0, -891.0, -843.0, -797.0, 0, 0, 0, 0, 0, 0]


In [ ]:
display(HTML(Q_TABLE.sort_values('MAX', ascending=False).head(100).to_html()))

,UP,DOWN,LEFT,RIGHT,MAX
41,13874.265000,0.000000,8926.355684,0.000000,13874.265000
30,0.000000,5891.697309,0.000000,13483.800000,13483.800000
19,12107.855684,4772.700000,0.000000,0.000000,12107.855684
40,2433.600000,0.000000,2391.982875,11798.728875,11798.728875
17,8260.980308,0.000000,658.800000,4004.862964,11772.181746
31,9000.000000,2908.800000,0.000000,4772.700000,11214.307125
32,7408.800000,0.000000,10630.372500,0.000000,10630.372500
29,10243.955684,0.000000,0.000000,4772.700000,10243.955684
42,0.000000,9909.534832,2433.600000,0.000000,9909.534832
28,8101.246708,0.000000,7284.046708,2433.600000,9581.302587
